In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)

%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from numpy.random import randn
from numpy import diag, ndarray



In [3]:
import sys
sys.path.append('..')

In [4]:
from utils.kronecker import KroneckerOperator, KroneckerProduct, KroneckerSum, KroneckerDiag, ProductChain, SumChain
from utils.linalg import kronecker_product_literal, kronecker_sum_literal, vec, ten


In [5]:
N1 = 6
N2 = 5
N3 = 4
N4 = 3

A1 = randn(N1, N1)
A2 = randn(N2, N2)
A3 = randn(N3, N3)
A4 = randn(N4, N4)

X = randn(N4, N3, N2, N1)
D = randn(N4, N3, N2, N1)

In [13]:
(kronecker_sum_literal(A1, A2, A3, A4) * kronecker_product_literal(A1, A2, A3, A4)) @ vec(X)

array([ 40.256, -10.571,  15.522,  21.423,   1.295,   0.52 ,  23.634,   3.935,   1.208,  31.896,   4.173,  -1.004,  -2.16 ,   0.802,   1.819,  16.229,   0.988,  -1.75 ,   5.495,   2.14 ,   1.096,  13.69 ,   2.677,   0.678,  11.437,   0.529,   6.522,  -1.98 ,  -1.79 ,   3.873, -38.655,  -2.881,  -2.228,  -6.45 ,  -2.546,   1.236,  15.236,  13.549,   6.285,  56.312,  -8.859,   3.384,  14.136,  -1.196,   0.504,  25.526,  -2.469,  -0.287, -51.481,  -0.162,  -6.447,  -1.354,   5.724,  -6.346,
         2.778,   7.909,  -1.978,  27.317,   0.54 ,  -1.809, -28.432,   2.402,  -3.287, -12.175,   4.078,  -3.274,   8.229,   3.091,  -0.849,   3.682,   1.225,   0.772,  -2.845,   0.595,  -0.168,   3.763,   0.314,   0.105,  -4.13 ,   0.702,   0.294,  -0.962,  -0.737,   0.542,  -1.007,   0.31 ,  -1.594,   5.84 ,   1.645,  -0.87 ,   0.747,   0.238,   0.055,   4.8  ,   0.774,   0.202,   1.298,  -2.452,  -1.977,   3.251,  -1.899,  -3.677,  -3.126,  -0.071,  -0.989,   0.485,  -1.084,   0.458,
        -0.203